In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE95843"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE95843"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE95843.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE95843.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE95843.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression from embryoid bodies derived from HBG3 HB9 ES cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: F1 progeny of two different strains (C57BL/6 and SJL)', 'strain: B6CBA-Tg(HDexon1)62Gpb/3J'], 1: ['time point: 9 weeks cultures', 'time point: 10 weeks cultures', "treatment: plasma from a Huntington's disease mouse model"]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and sample characteristics, it appears this dataset contains
# information about amyloid beta levels, but not gene expression data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary, we don't see any clear information about 
# Huntington's Disease, age, or gender

# 2.1 Data Availability
trait_row = None  # No information about Huntington's Disease in the samples
age_row = None    # No age information available
gender_row = None # No gender information available

# 2.2 Data Type Conversion (defining functions even though data is unavailable)
def convert_trait(value):
    """Convert trait value to binary (0 for control, 1 for Huntington's Disease)"""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    if "disease" in value or "patient" in value or "case" in value or "huntington" in value or "hd" in value:
        return 1
    elif "control" in value or "healthy" in value or "normal" in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous"""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    if "female" in value or "f" == value.strip():
        return 0
    elif "male" in value or "m" == value.strip():
        return 1
    return None

# 3. Save Metadata
# We'll set is_trait_available to False as we couldn't find trait information
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction - Skip this step as trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE95843/GSE95843-GPL23148_series_matrix.txt.gz


Gene data shape: (19620, 65)
First 20 gene/probe identifiers:
Index(['a', 'A030009H04Rik', 'A130010J15Rik', 'A130023I24Rik', 'A1bg', 'A1cf',
       'A230006K03Rik', 'A230046K03Rik', 'A230050P20Rik', 'A230051G13Rik',
       'A230051N06Rik', 'A230083G16Rik', 'A2ld1', 'A2m', 'A330021E22Rik',
       'A330049M08Rik', 'A330070K13Rik', 'A3galt2', 'A430005L14Rik',
       'A430033K04Rik'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the provided output, the gene identifiers appear to be human gene symbols.
# These are standard gene symbols like A1BG, A2M, AAAS, etc. that match official human gene nomenclature.
# They are not probe IDs (which would typically be numeric or have manufacturer prefixes)
# and they are not other types of identifiers that would require mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# Based on previous steps, this dataset does not contain Huntington's Disease trait information
# and trait_row was found to be None, so we need to adjust our approach

# 1. First, normalize the gene symbols in the gene data we extracted in Step 3
print("Normalizing gene symbols...")
# Make sure output directory exists
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Normalize gene symbols directly from gene_data variable obtained in Step 3
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 normalized gene symbols:")
print(normalized_gene_data.index[:10])

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Since we don't have trait data, we need to prepare a proper dataframe for validation
# and set is_biased=True since without trait data, the dataset is biased/unusable for trait analysis
dummy_df = normalized_gene_data.iloc[:5, :5].reset_index()  # Create small sample for efficiency
is_biased = True  # Without trait data, the dataset is considered biased/unusable

print("\nValidating cohort usability...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,  # We did find gene expression data
    is_trait_available=False,  # Previous steps found no trait information
    is_biased=is_biased,  # Dataset is biased without trait data
    df=dummy_df,  # Use a small sample of the data for validation
    note="This dataset contains gene expression data but lacks Huntington's Disease trait information."
)

print(f"Dataset usability: {is_usable}")
print("Dataset lacks trait information for Huntington's Disease, so no linked data will be saved.")

Normalizing gene symbols...
Gene data shape after normalization: (14859, 65)
First 10 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS', 'AACS',
       'AADAC', 'AADACL3'],
      dtype='object', name='ID')


Normalized gene data saved to: ../../output/preprocess/Huntingtons_Disease/gene_data/GSE95843.csv

Validating cohort usability...
Dataset usability: False
Dataset lacks trait information for Huntington's Disease, so no linked data will be saved.
